In [1]:
import pandas as pd
import requests
import json
import re
import os
from tqdm import tqdm
import random
import xml.etree.ElementTree as ET


key = 'QypW6JzQ4XfGViss1Ks4g'

user_agents = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 13_1) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15'
]

def select_agent():
    n=random.randint(0,len(user_agents)-1)
    return user_agents[n]

In [2]:
df = pd.read_csv('banned_book_data_combined_lists.csv')
print(df.shape)

(55278, 8)


In [3]:
genres = dict()
titles = dict()

directory = 'gr_shelves'


# Check whether the specified path exists or not
isExist = os.path.exists(directory)
if not isExist:
    os.makedirs(directory)
    print("The new directory is created!")


with open('titles.csv','w',encoding='utf-8') as out:
    for i,row in df.iterrows():
        path = f'{directory}/{i}.txt'
        if not(os.path.isfile(path)):
            out = open(f'{directory}/{i}.txt','w',encoding='utf-8')
            user_shelves = []
            title = row['publication']
            title = re.sub(r'\s+','+',title)
            url = f'https://www.goodreads.com/book/title.xml?key={key}&title={title}'
            print(url)
            headers = {'User-Agent': select_agent() }
            response = requests.get(url, headers=headers)
            if response:

                xml_data = response.text
                root = ET.fromstring(xml_data)
                book = root.find('book')
                out.write(book.find('id').text)
                out.write('\n')
                out.write(row['publication'])
                out.write('\n')
                out.write(book.find('title').text)
                out.write('\n')
                shelves = book.findall('popular_shelves/shelf')
                for shelf in shelves:
                    #print(shelf.get('name'))
                    out.write(f"{shelf.get('name')}\n")

            out.close()


In [4]:
# # Convert and write JSON object to file
# with open("genres_gr.json", "w") as outfile: 
#     json.dump(genres, outfile)